In [ ]:
import random #for randomization of ship placement and other things
import numpy as np #for various array operations
#import pandas as pd

number_of_ships = 1 #number of ships that will be initialized on the board
ship_len = 3 #length of every ship. this will need to be replaced for varying length ships
battleship_array_len = 5 #dimensions of the battleship board. 5 = 5x5 board

battleship_array = [] #array representing the battleship board that will be guessed on
battleship_cheat = [] #'cheat sheet' array containing the positions of all ships on the board

def initialize_arrays(static_answer = False): #randomize battleship placement in 1d cheat array, make guess array
    global battleship_array
    global battleship_cheat  
    battleship_array = []
    battleship_cheat = []
    #make battleship guess array   
    for i in range(battleship_array_len):
        row = []
        for j in range(battleship_array_len):
            row.append('?') #board spaces unknown to attacking player at beginning of game

        battleship_array.append(row)

    if static_answer == False: #randomly place ships on board. needs to be modified to place more than one ship
        #make battleship answer array  
        ship_placement_x = random.randint(0, battleship_array_len - ship_len)
        ship_placement_y = random.randint(0, battleship_array_len - ship_len)
        orientation = random.randint(0, 1) #0 is horizontal (right) 1 is vertical (down)
        #orientation = 0
        if orientation == 0: #horizontal              
            for i in range(battleship_array_len):
                row = []
                for j in range(battleship_array_len):
                    if i == ship_placement_y and j >= ship_placement_x and j < ship_placement_x + ship_len:
                        #'O' is a piece of a ship. this needs to be expanded for different ship types and unsunk ships
                        row.append('O') 
                    else:
                        row.append('x') #'x' is an empty space, aka a miss
                battleship_cheat.append(row)
        if orientation == 1: #vertical              
            for i in range(battleship_array_len):
                row = []
                for j in range(battleship_array_len):
                    if j == ship_placement_x and i >= ship_placement_y and i < ship_placement_y + ship_len:
                        row.append('O')
                    else:
                        row.append('x')
                battleship_cheat.append(row)
    if static_answer == True: #places ships on board with hard coded coords. needs to be modified to place more than one ship
        #make battleship answer array  
        ship_placement_x = 2
        ship_placement_y = 2
        orientation = 0 #0 is horizontal (right) 1 is vertical (down)
        #orientation = 0
        if orientation == 0: #horizontal              
            for i in range(battleship_array_len):
                row = []
                for j in range(battleship_array_len):
                    if i == ship_placement_y and j >= ship_placement_x and j < ship_placement_x + ship_len:
                        row.append('O')
                    else:
                        row.append('x')
                battleship_cheat.append(row)
        if orientation == 1: #vertical              
            for i in range(battleship_array_len):
                row = []
                for j in range(battleship_array_len):
                    if j == ship_placement_x and i >= ship_placement_y and i < ship_placement_y + ship_len:
                        row.append('O')
                    else:
                        row.append('x')
                battleship_cheat.append(row)
    #print(orientation)
    #print(ship_placement_x)
    #print(ship_placement_y)
                  
initialize_arrays()

def print_array(array): #prints a battleship array out in ascii format
    print('\n'.join([''.join(['{:4}'.format(item) for item in row]) 
      for row in array]))
    print() 


print_array(battleship_array)
print_array(battleship_cheat)

In [ ]:
def initQTable(lengthOfBoard):
    # Takes in the length of the board and returns a two dimensional array
    # to be used for qtable, with rows = {Start, Miss (board coords), Hit (board coords)} 
    #and cols = coords of board (actions)
    
    numStates = 3 #Start, Miss, Hit
    numPrevCalls = 2 #Number of previous calls accounted for in Q-Table (1 Hit, 1 Miss)
    actionSize = lengthOfBoard**2  
    stateSize = numPrevCalls*actionSize+1
    qtable = np.zeros((stateSize, actionSize))
    #df=pd.DataFrame(qtable)
    
    return qtable
  

In [ ]:
qtable1 = initQTable(5)
qtable1.shape

In [ ]:
# def initQTable(lengthOfBoard):
#     # Takes in the length of the board and returns a two dimensional array
#     # to be used for qtable, with rows = {Start, Miss (board coords), Hit (board coords)} 
#     #and cols = coords of board (actions)
    
#     numStates = 3 # Start, Miss, Hit
#     qtable = []
#     numCells = lengthOfBoard**2
#     for i in range(2*numCells+1):
#         qtable.append([0]*numCells)
#     return qtable

# a = battleship_array_len 
# for row in initQTable(a):
#     print(row)
# #print(initQTable(a))
# qtable = initQTable(battleship_array_len)
# len(qtable)


In [ ]:
repeat_penalty = -1 #code corresponding to calling an already called space. obsolete since repeat hits were removed as an option
miss_penalty = 0 #code corresponding to missing on a call
hit_reward = 1 #code corresponding to hitting a ship on a call
sink_bonus = 2 #code corresponding to sinking a ship on a call. the entire system will have to be reworked with multiple ships

result_dict = {
  repeat_penalty: "already called",
  miss_penalty: "miss",
  hit_reward: "hit",  
}

result_reward_dict = { #q-table rewards/penalties associated with each possible result of an action. 
  repeat_penalty: 0,
  miss_penalty: 0,
  hit_reward: 10,  
}    

def check_guess(guess_x, guess_y): #assigns q-table reward values to various results of actions
    #will definately need to be reworked once the board becomes more complex
    global battleship_array
    global battleship_cheat 
    
    if battleship_cheat[guess_x][guess_y] == 'x' and battleship_array[guess_x][guess_y] == '?': #miss
        battleship_array[guess_x][guess_y] = 'x'
        return miss_penalty
        
    if battleship_cheat[guess_x][guess_y] == 'x' and battleship_array[guess_x][guess_y] == 'x': #already called miss
        return repeat_penalty
        
    if battleship_cheat[guess_x][guess_y] == 'O' and battleship_array[guess_x][guess_y] == '?': #hit
        battleship_array[guess_x][guess_y] = 'O'
        return hit_reward
        
    if battleship_cheat[guess_x][guess_y] == 'O' and battleship_array[guess_x][guess_y] == 'O': #already called hit
        return repeat_penalty
        
    return -1000 #array mismatch or something went really wrong
        
print_guess_results = True #prints result of guesses made (hit, miss)
print_arrays = True #prints the board and cheat arrays  
    
def make_guesses(): #plays out a solitaire game of battleship randomly guessing coords
    total_hits = 0 #total number of hits currently on board
    turn = 0 #turn counter
    turn_limit = 100
    global battleship_array_len
    global ship_len
    global battleship_array
    global battleship_cheat
    already_called = [] #keeps track of coords that have already been called
    
    initialize_arrays()
    if print_arrays: #starting state. print out initial arrays
        print("begin")
        print_array(battleship_array)
        print("Cheat Sheet:")
        print_array(battleship_cheat)
        print("-------")
    while total_hits < number_of_ships * ship_len and turn < turn_limit: #what happens on each turn of the game
        turn += 1
        
        #make random guess
        valid_guess = False
        while not valid_guess: #make random guess
            guess_x = random.randint(0, battleship_array_len - 1)
            guess_y = random.randint(0, battleship_array_len - 1)
            if [guess_x, guess_y] not in already_called: #make sure it is not a repeat guess
                already_called.append([guess_x, guess_y])
                valid_guess = True
        
        
        #check result of guess
        result = check_guess(guess_x, guess_y) #hit, miss, or already called
        
        if print_arrays: #resulting arrays after guess
            print("Turn", turn)
            print_array(battleship_array)
            print("Cheat Sheet:")
            print_array(battleship_cheat) 
            print("-------")
        if result_dict[result] == "hit": #hit
            total_hits += 1 
    return turn
    
turns = make_guesses()
print("Finished in", turns, "turns.")

In [ ]:
#20000 runs of randomly guessing
print_guess_results = False
print_arrays = False
runs = 20000
turn_count_array = [] #keep track on turns taken on every run
for i in range(runs):
    #initialize_arrays()
    turn_count = make_guesses() #play one game
    turn_count_array.append(turn_count) #append to the turn count array
turn_mean = np.mean(turn_count_array)
turn_std = np.std(turn_count_array)
print("Strategy: Randomly Guessing")
print(runs, "games")
print("Average turns taken:", turn_mean)
print("Stdev of turns:", turn_std)
#print((turn_count_array))

In [ ]:
#Define hyperparameters for RL 
runs = 80000
learning_rate = 0.8 #currently unused
max_moves = 100
gamma = 0.95 # discounting rate, currently unused

epsilon = 1 #controls explore vs exploit
max_epsilon = 1
min_epsilon = 0.01

decay_rate = 0.00001 #used to decrement epsilon

In [ ]:
def action_to_coord(action): #converts a number to the corresponding coordinate (e.g. '5 = (0, 4)', '6 = (1, 0)' (array len 5))
    #janky way to do this, and will be obsolete when the q-table gets put into a better format
    action_num = action
    guess_x = 0
    guess_y = 0
    
    while action_num > 0:
        if action_num >= battleship_array_len: #counting y coord
            guess_x += 1
            action_num -= battleship_array_len
        else: #counting x coord
            guess_y += 1
            action_num -= 1
    return guess_x, guess_y

#1000 runs of learning/training
print_guess_results = False
print_arrays = False
turn_count_array = []
qtable = initQTable(battleship_array_len)


for run in range(runs):
    global already_called
    initialize_arrays(static_answer = False)
    if print_arrays: #starting state
        print("begin")
        print_array(battleship_array)
        print("Cheat Sheet:")
        print_array(battleship_cheat)
        print("-------")
    
    turn = 0
    state = 0 #start
    total_hits = 0  
    already_called = []
    
    while total_hits < number_of_ships * ship_len and turn < max_moves: #each turn, check if the game should end (needs rework)
        
    #for turn in range(max_moves):
        # choose an action in the current state (explore or exploit)
        choose_mode = random.uniform(0,1)
        
        
        valid_guess = False
        # if choice > epsilon, --> exploitation (take biggest Q value for state)
        if choose_mode > epsilon:
        #if True:
            qmax = float('-inf')
            action = -1 #actions correspond to the spots on the battleship board
            for i in range(battleship_array_len**2 - 1):
                guess_x, guess_y = action_to_coord(i)
                #check for highest Q value action given the current state
                if qtable[state][i] > qmax and [guess_x, guess_y] not in already_called:
                    action = i
                    qmax = qtable[state][i]
            guess_x, guess_y = action_to_coord(action)
            already_called.append([guess_x, guess_y])
                    
        #else random choice --> exploration
        else:
            while not valid_guess:
                action = random.randint(0, battleship_array_len**2-1) #action possibilities are the board size squared
                guess_x, guess_y = action_to_coord(action)
                if [guess_x, guess_y] not in already_called:
                    already_called.append([guess_x, guess_y])
                    valid_guess = True
        
        #take action and update the states and rewards
        """
        # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
        # qtable[new_state,:] : all the actions we can take from new state
        qtable[state, action] = qtable[state, action] + learning_rate * (reward + gamma * np.max(qtable[new_state, :]) - qtable[state, action])
        """
        
        #take action(make guess)
        guess_x, guess_y = action_to_coord(action) #take the action chosen by the agent, convert it to the coord to be called
        #print("Action:", action)
        
        result = check_guess(guess_x, guess_y) #hit, miss, or already called
        #print("Next guess:", guess_x, guess_y)
        #print("Result:", result)
        
        if result == hit_reward and total_hits == number_of_ships * ship_len - 1: #sunk ship, more reward
            #this should probably be done in a better way?
            qtable[state][action] += result_reward_dict[result] + sink_bonus
        else:
            qtable[state][action] += result_reward_dict[result]
        

        if result_dict[result] == "hit":
            state = action+1 #next state corresponds to the action just taken and result
            total_hits += 1 
        else:
            if result_dict[result] == "miss":
                state = action + battleship_array_len**2 + 1 #next state corresponds to the action just taken and result
        turn += 1               
    #reduce epsilon (less exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*run)
    #print("Total hits:", total_hits)
    turn_count_array.append(turn) #keep track of turns taken in each game
                   
turn_mean = np.mean(turn_count_array)
turn_std = np.std(turn_count_array)
print(runs, "games")
print("Average turns taken:", turn_mean)
print("Stdev of turns:", turn_std)

#below is an example of the methodology behind action_to_coord. 5 = (1, 2)
"""
-------------
| 0 | 1 | 2 |
-------------
| 3 | 4 | 5 |
-------------
| 6 | 7 | 8 |
-------------
"""

In [ ]:
#display q table after training. kinda messy, ideally we would have some kind of image representation
for row in qtable:
    print(row) 

In [ ]:
#testing loop
#1000 runs of testing
#same as above loop, but we only exploit (use Q-table)
print_guess_results = False
print_arrays = False
turn_count_array = []

for i in range(int(runs/4)):
    global qtable
    initialize_arrays(static_answer = False)
    if print_arrays: #starting state
        print("begin")
        print_array(battleship_array)
        print("Cheat Sheet:")
        print_array(battleship_cheat)
        print("-------")
    
    turn = 0
    state = 0 #start
    total_hits = 0
    already_called = []
    
    while total_hits < number_of_ships * ship_len and turn < max_moves:
        if True: #always exploit (use highest Q-table values) during testing
            qmax = float('-inf')
            action = -1
            for i in range(battleship_array_len**2 - 1):
                guess_x, guess_y = action_to_coord(i)
                if qtable[state][i] > qmax and [guess_x, guess_y] not in already_called:
                    action = i
                    qmax = qtable[state][i]
            guess_x, guess_y = action_to_coord(action)
            already_called.append([guess_x, guess_y])
                        
        #print(already_called)
        
        #take action and update the state

        
        #take action(make guess)
        guess_x, guess_y = action_to_coord(action)
        #print("State:", state)
        #print("Action:", action)
        
        result = check_guess(guess_x, guess_y) #hit, miss, or already called
        #print("Next guess:", guess_x, guess_y)
        #print("Result:", result)

        if result_dict[result] == "hit":
            state = action+1
            total_hits += 1 
        else:
            if result_dict[result] == "miss":
                state = action + battleship_array_len**2 + 1               
        turn += 1        
    already_called = []        
    #print("Total hits:", total_hits)
    turn_count_array.append(turn)
                   
turn_mean = np.mean(turn_count_array)
turn_std = np.std(turn_count_array)
print("Strategy: Q-table Reinforcement Learning")
print(int(runs/4), "games")
print("Average turns taken:", turn_mean)
print("Stdev of turns:", turn_std)

In [ ]:
for row in qtable:
    print(row)

In [ ]:
#see how trained q-table works in one game
print_guess_results = True
print_arrays = True
print_guess_results = False
print_arrays = False
turn_count_array = []

for i in range(1): #one game
    global qtable
    initialize_arrays(static_answer = False)
    if print_arrays: #starting state
        print("begin")
        print_array(battleship_array)
        print("Cheat Sheet:")
        print_array(battleship_cheat)
        print("-------")
    #turn_count = make_guesses()
    #turn_count_array.append(turn_count)
    
    turn = 0
    state = 0 #start
    total_hits = 0
    already_called = []
    
    while total_hits < number_of_ships * ship_len and turn < max_moves:
        if True: #always exploit during testing
            qmax = float('-inf')
            action = -1
            for i in range(battleship_array_len**2 - 1):
                guess_x, guess_y = action_to_coord(i)
                if qtable[state][i] > qmax and [guess_x, guess_y] not in already_called:
                    action = i
                    qmax = qtable[state][i]
            guess_x, guess_y = action_to_coord(action)
            already_called.append([guess_x, guess_y])
                  
        
        #print(already_called)
        
        #take action and update the states and rewards
        
        #take action(make guess)
        guess_x, guess_y = action_to_coord(action)
        #print("State:", state)
        #print("Action:", action)
        
        print("Turn:", turn)
        print_array(battleship_array)
        print("Cheat Sheet:")
        print_array(battleship_cheat)
        result = check_guess(guess_x, guess_y) #hit, miss, or already called
        #print("Next guess:", guess_x, guess_y)
        #print("Result:", result)
        

        if result_dict[result] == "hit":
            state = action+1
            total_hits += 1 
        else:
            if result_dict[result] == "miss":
                state = action + battleship_array_len**2 + 1               
        turn += 1        
    already_called = []        
    #reduce epsilon (less exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*i)
    #print("Total hits:", total_hits)
    turn_count_array.append(turn)
    
print("victory")
print_array(battleship_array)
print("-------")                   
turn_mean = np.mean(turn_count_array)
turn_std = np.std(turn_count_array)
print("Strategy: Q-table Reinforcement Learning")
print(int(1), "games")
print("Turns taken:", turn_mean)


In [ ]:
#human player
print_guess_results = True
print_arrays = True
turn_count_array = []

for i in range(1): #one game
    global qtable
    initialize_arrays(static_answer = False)
    if print_arrays: #starting state
        print("begin")
        print_array(battleship_array)
        print("-------")
    
    turn = 0
    total_hits = 0
    already_called = []
    
    while total_hits < number_of_ships * ship_len and turn < max_moves:

        
        print("Turn:", turn)
        print_array(battleship_array)
        guess_y = int(input("Enter x-coord: ")) #prompt user for x-coord guess
        guess_x = int(input("Enter y-coord: ")) #prompt user for y-coord guess
        result = check_guess(guess_x, guess_y) #hit, miss, or already called
        print(result_dict[result]) #tell user what the result of their call was
        if result_dict[result] == "hit":
            total_hits += 1 
            
        turn += 1        
    already_called = []  
    #print("Total hits:", total_hits)
    turn_count_array.append(turn)

print("victory")
print_array(battleship_array)
print("-------")
turn_mean = np.mean(turn_count_array)
turn_std = np.std(turn_count_array)
print("Strategy: Human Player")
print(int(1), "games")
print("Turns taken:", turn_mean)